In [9]:
import secrets
from collections import deque

class Trivium:
    def __init__(self, key, iv):
        self.state = None
        self.key = key
        self.iv = iv
        
        # Khởi tạo trạng thái ban đầu với key và iv
        init_state = list(map(int, list(self.key))) + [0] * 13
        init_state += list(map(int, list(self.iv))) + [0] * 4
        init_state += [0] * 108 + [1, 1, 1]
        self.state = deque(init_state)

        # Khởi tạo Trivium với 4 vòng lặp
        for _ in range(4 * 288):
            self._generate_keystream()

    def _generate_keystream(self):
        t1 = self.state[65] ^ self.state[92]
        t2 = self.state[161] ^ self.state[176]
        t3 = self.state[242] ^ self.state[287]

        z = t1 ^ t2 ^ t3

        t1 = t1 ^ self.state[90] & self.state[91] ^ self.state[170]
        t2 = t2 ^ self.state[174] & self.state[175] ^ self.state[263]
        t3 = t3 ^ self.state[285] & self.state[286] ^ self.state[68]

        self.state.rotate()

        self.state[0] = t3
        self.state[93] = t1
        self.state[177] = t2

        return z

    def generate_keystream(self, msg_length):
        # Tạo ra keystream có độ dài bằng số bit của plaintext
        keystream = []
        for _ in range(msg_length):
            keystream.append(self._generate_keystream())
        return self.bits_to_hex(keystream)

    def encrypt(self, message, keystream):
        # Hàm giải mã với keystream và ciphertext bằng hàm XOR
        keystream = self.hex_to_bytes(keystream)
        cipher = bytearray()
        for i in range(len(keystream)):
            cipher.append(message[i] ^ keystream[i] & 0xff)
        return cipher

    def decrypt(self, cipher, keystream):
        # Hàm này hoạt động với các chuỗi và file .txt
        # Hàm giải mã với keystream và ciphertext bằng hàm XOR
        keystream = self.hex_to_bytes(keystream)
        text = bytearray()
        for i in range(len(keystream)):
            text.append(cipher[i] ^ keystream[i] & 0xff)
        return text

    @staticmethod
    def hex_to_bytes(hex_string):
        byte_list = []
        for i in range(0, len(hex_string), 2):
            byte = int(hex_string[i:i+2], 16)
            byte_list.append(byte)
        return byte_list

    @staticmethod
    def bits_to_hex(bits):
        hex_string = ""
        for i in range(0, len(bits), 8):
            byte_sum = sum([bits[i + j] << j for j in range(8)])
            hex_string += "%02X" % byte_sum
        return hex_string
    
def generate_random_bits(length):
    rand_bits = secrets.randbits(length)
    rand_string = '{0:080b}'.format(rand_bits)
    return bytearray(map(int, rand_string))


def read_bytes_from_file(filename):
    with open(filename, "rb") as file:
        return file.read()

def encrypt(input_file, output_file):
    key = generate_random_bits(80)
    iv = generate_random_bits(80)
    plaintext = read_bytes_from_file(input_file)

    trivium = Trivium(key, iv)
    keystream = trivium.generate_keystream(len(plaintext) * 8)

    print("IV (in hex):", trivium.bits_to_hex(iv))
    print("Key (in hex):", trivium.bits_to_hex(key))
    print("Keystream (in hex):", keystream)

    ciphertext = trivium.encrypt(plaintext, keystream)

    print("Cipher (in hex):", ciphertext.hex())

    with open(output_file + "_binary.txt", "wb") as file:
        file.write(iv)
        file.write(ciphertext)

    with open(output_file + "_hex.txt", "w") as file:
        file.write(iv.hex()+'\n')
        file.write(ciphertext.hex())

    print("Decryption Result:", trivium.decrypt(ciphertext, keystream).decode())

def main():
    input_file = r"C:\Users\ADMIN\Downloads\Test Data\alice29.txt"
    output_file = "encrypted"
    encrypt(input_file, output_file)

if __name__ == "__main__":
    main()


IV (in hex): D7B82422A1F1CEF37656
Key (in hex): 8E2920DD1B6CE7D17B2B
Keystream (in hex): ...
Cipher (in hex): ...
Decryption Result: 

  Alice was beginning to get very tired of sitting by her sister
on the bank, and of having nothing to do:  once or twice she had
peeped into the book her sister was reading, but it had no
pictures or conversations in it, `and what is the use of a book,'
thought Alice `without pictures or conversation?'

  So she was considering in her own mind (as well as she could,
for the hot day made her feel very sleepy and stupid), whether
the pleasure of making a daisy-chain would be worth the trouble
of getting up and picking the daisies, when suddenly a White
Rabbit with pink eyes ran close by her.

  There was nothing so VERY remarkable in that; nor did Alice
think it so VERY much out of the way to hear the Rabbit say to
itself, `Oh dear!  Oh dear!  I shall be late!'  (when she thought
it over afterwards, it occurred to her that she ought to have
wondered at t

### Tiến hành mã hóa trên các file trong thư mục Test Data, lưu file mã hóa dạng binary và Hex vào thư mục Test Data Encrypted

In [8]:
import os
import time
def encrypt_no_print(input_file, output_file):
    key = generate_random_bits(80)
    iv = generate_random_bits(80)
    plaintext = read_bytes_from_file(input_file)

    trivium = Trivium(key, iv)
    keystream = trivium.generate_keystream(len(plaintext) * 8)

    # print("IV (in hex):", trivium.bits_to_hex(iv))
    # print("Key (in hex):", trivium.bits_to_hex(key))
    # print("Keystream (in hex):", keystream)

    ciphertext = trivium.encrypt(plaintext, keystream)

    # print("Cipher (in hex):", ciphertext.hex())

    with open(output_file + "_binary.txt", "wb") as file:
        file.write(iv)
        file.write(ciphertext)

    with open(output_file + "_hex.txt", "w") as file:
        file.write(iv.hex()+'\n')
        file.write(ciphertext.hex())

def run():
    input_directory = r"C:\Users\ADMIN\Downloads\Test Data"
    output_directory = r"C:\Users\ADMIN\Downloads\Test Data encrypted"

    # Tạo thư mục đầu ra nếu nó chưa tồn tại
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Lặp qua các tệp trong thư mục đầu vào
    for filename in os.listdir(input_directory):

        input_file = os.path.join(input_directory, filename)
        output_file = os.path.join(output_directory, filename + 
                                   "_encrypted")

        start_time = time.time()
        encrypt_no_print(input_file, output_file)
        end_time = time.time()
        file_size = os.path.getsize(input_file)
        print("File:", filename, "- Size:", file_size, "bytes", 
              "- Time:", round(end_time - start_time, 2), "seconds")

run()

File: alice29.txt - Size: 151842 bytes - Time: 6.8 seconds
File: asyoulik.txt - Size: 125179 bytes - Time: 5.67 seconds
File: bible.txt - Size: 4047392 bytes - Time: 182.07 seconds
File: cantrbry.zip - Size: 733976 bytes - Time: 33.03 seconds
File: cp.html - Size: 24603 bytes - Time: 1.06 seconds
File: E.coli - Size: 4638690 bytes - Time: 211.29 seconds
File: fields.c - Size: 11150 bytes - Time: 0.47 seconds
File: grammar.lsp - Size: 3721 bytes - Time: 0.17 seconds
File: kennedy.xls - Size: 1029744 bytes - Time: 44.82 seconds
File: large.zip - Size: 3256280 bytes - Time: 148.2 seconds
File: lcet10.txt - Size: 426754 bytes - Time: 18.79 seconds
File: plrabn12.txt - Size: 481861 bytes - Time: 20.79 seconds
File: ptt5 - Size: 513216 bytes - Time: 22.08 seconds
File: sum - Size: 38240 bytes - Time: 1.72 seconds
File: world192.txt - Size: 2473400 bytes - Time: 108.05 seconds
File: xargs.1 - Size: 4227 bytes - Time: 0.19 seconds
